In [ ]:
import random
import warnings
warnings.filterwarnings("ignore")
import math
import os
import time
import random
import shutil
import numpy as np
from scipy import spatial as st 
import pandas as pd
import timeit


#####################################################################################
#####################################################################################
#############-----------variable initialization----------###################
                       # Initialize basic variables #
# this is the name of the folder with the data
mainFolder = "datafile"
if os.path.exists(mainFolder) and os.path.isdir(mainFolder):
    shutil.rmtree(mainFolder)

# range of values for beta (heterogeneity)
betas = [ 1.5, 2.5, 3.5, 4.5 ]

# range of values for the forager population (tested against every beta)
foragers = [ 1, 5, 10, 50, 100, 200 ]

# number of files created for each program with different initial conditions 
# helps calculate averages
runs = 1

# number of time-steps for each run
steps = 100

# number of patches per run 
patches = 50000 

# number of patches in the memory of each forager at t = 0
# recall that memory > time-steps and memory <= patches, otherwise we get an error
memory = 50000 

## foraging radius
## whole landscape ranges from 0 to 1
fRadius = [1, 0.1, 0.01, 0.001]
global all_movdis
all_movdis =[]

##effective radius for moving to a new home location
er = 2


In [ ]:
class Grid:
    """This class represents the grid where patches and foragers interact"""

    def __init__(self, foragers, memory, patches, beta, kmax):
        """This is what builds the grid. Called every time a new grid is constructed."""

        #print ("grid created")
        # initialize the variables
        self.numForagers = foragers
        self.memory = memory      
        self.numPatches = patches
        self.beta = beta
        self.kmax = kmax
        self.meanK = 0
        self.bagPatches=[]
        self.bagForagers = []
        

        # create patches
        self.createPatches()
        
        # create foragers
        self.createForagers()
        #----------------------------------------------------------------------------------------------------------

    def nextState(self,rad):
        """this function is in charge of changing the foragers and tress to a new state every new time-step"""
        # sumK = 0
        z = 0


        # loop for the next state of the patches
        while z < self.numPatches:
            self.bagPatches[z].nextState()

                
            # Add number of foods to meanK counter
            z = z + 1
        z = 0
        


        # loop for the next state of the foragers
        while z < self.numForagers:
            #print("forager", z)
            self.bagForagers[z].nextState(rad)
            z = z + 1
        z = 0
        string = ""
        
        while z < len(self.bagForagers):
            string = string + str(self.bagForagers[z].name)+","+ str(self.bagForagers[z].x)+","+str(self.bagForagers[z].y)+","+str(self.bagForagers[z].patch.k)+","
            
            z = z + 1
        return string+"\n"
    
    def output(self):
        string_dis = ""
        z = 0 
        while z < len(self.bagForagers):
            string_dis = string_dis + str(self.beta) +","+ str(self.bagForagers[z].name)+","+ str(self.bagForagers[z].dismoved)+","+ str(self.bagForagers[z].homemoved) +","+str(self.bagForagers[z].totalfoodinrad) +","+str(self.bagForagers[z].logmoves) +","+str(self.bagForagers[z].resmoves)+"\n"
#             #print(string_dis)
            z = z + 1
        return string_dis+"\n"
        #----------------------------------------------------------------------------------------------------------  
            
    def createPatches(self):
        '''This method is in charge of creating the patches according to a beta distribution'''
        z = 1
        c = 0

        patch = 0 
        # patch counter

        # for the variable c from k=1 to k= kmax, we need to iterate
        while z <= self.kmax:
            c = c + pow(z, -self.beta)
            z = z +1
            
        # Loop for patch creating that is repeated until all patches are created
        while len(self.bagPatches) < self.numPatches:
            k = 1
     
            # try to create a patch for all distribution from k to kmax
       
            # there's a problem with k = 1
            while (k <= self.kmax)&( len(self.bagPatches) < self.numPatches):
                c = c + pow(k, -self.beta)
                pk = pow(k, -self.beta)/c
                z = 0
     
                # try a certain amount of times to create a patch from a specific k 
                while (z <self.numPatches )&( len(self.bagPatches) < self.numPatches) :
                    z = z + 1
                    azar = random.random()
                
                    # if the probability is enough, then the patch is created
                    if azar < pk:
                        self.bagPatches.append(Patch(random.random(),random.random(), k, patch))
                        patch = patch + 1
                k = k + 1
    #----------------------------------------------------------------------------------------------------------
        
    def createForagers(self):
        """This method creates a certain amount of foragers, randomly distributed, a set of x,y coordinates, and a memory k"""
        z = 0
        dismoved = []
        homemoved = []
        foodinrad = 0
        totalfoodinrad = []
        # el loop de ceaccion
        # forager creation loop
        while z < self.numForagers :
            self.bagForagers.append(Forager( z , self.givePatchesMemory(), self, dismoved, homemoved,foodinrad, totalfoodinrad))
            z = z + 1
        #----------------------------------------------------------------------------------------------------------

    def givePatchesMemory(self):
        "This method generates and returns a set of memory patches for a forager"

        # fill container of size equal to self.memory
        memories = []

        # obtain a copy of the set of patches to which we can substract the chosen patches
        bagPatches2 = self.bagPatches[:]
        while len(memories)  < self.memory:
            # pick a random patch
            azar = random.randint(0 , (len(bagPatches2)-1))
            # turn it into memory 
            memories.append((bagPatches2[azar].x,bagPatches2[azar].y,bagPatches2[azar].k))
            # remove it from the set of patches 
            del bagPatches2[azar]
            

        return memories
    
    def givePatchEnPos(self, x, y):

        # This method helps locate a patch object based on its coordinates: x, y and k
        
        z=0
        # Search comparing each patch in the set of patches with the given coordinates
        while z < len(self.bagPatches):
            if (self.bagPatches[z].x == x) & (self.bagPatches[z].y == y):
                return self.bagPatches[z]
            z = z + 1

In [ ]:
class Forager:
    """This class represents a forager with all its attributes and functions"""

    def __init__(self, name, memories, grid, dismoved, homemoved, foodinrad, totalfoodinrad):
        '''Create a forager in a specific position, with a name, and a memory'''
        
        # initialize variables 
        self.name = name
        self.memories = memories
        self.grid = grid
        azar = random.randint(0, (len(self.memories) - 1 ))
        self.patch = self.grid.givePatchEnPos(self.memories[azar][0],self.memories[azar][1])
        self.homePatch = self.patch
        self.x = self.patch.x
        self.y = self.patch.y
        self.removeMemory(self.patch)
        self.patch.putGuest()
        self.kfRadius = 0 
#         self.kfRadius = self.check_radius_forfood()
        self.dismoved = []
        self.homemoved = []
        self.totalfoodinrad = []
        self.foodinrad = 0
        self.resmoves = 0
        self.logmoves = 0 
        self.previousheading = None
#         ##print(vars(self))
        #----------------------------------------------------------------------------------------------------------

    def nextState(self, rad):
        """Make sure they all start from homePatch"""
        
#         print(self.x, self.y, self.previousheading)
        r = rad
        homePatch = self.homePatch
        patch = self.patch
        self.x = patch.x
        self.y = patch.y
        if self.patch.k <= 0 :
            self.removeMemory(self.patch)
#             start = timeit.default_timer()
            self.kfRadius = self.check_radius_forfood(r)
#             print(timeit.default_timer() - start)
        # See food availavility within your fRadius
#             print("food in radius?", self.kfRadius)
           
        # If there is any food within your fRadius, you forage
            if self.kfRadius != None:
                # change coordinates
                self.x = self.kfRadius[0]
                self.y = self.kfRadius[1]
                self.foodinrad = self.foodinrad + 1
    #             ##print(self.foodinrad)
                # move from the old patch and get to the new one
                patch = self.grid.givePatchEnPos(self.x , self.y)
                self.patch.removeGuest()
                patch.putGuest()
                self.dismoved.append(round(self.kfRadius[2],5))
                # Here DO NOT remove patch (as not depleted) from memory
                self.patch = patch
                self.logmoves += 1 

            # If no food within your fRadius, you change homePatch
            else : 
                # Examine distances and Ks 
                self.totalfoodinrad.append(self.foodinrad)
                self.foodinrad = 0
                self.x, self.y, d = self.check_forhome_avoidoverlap(r)
                self.kfRadius = 0
                self.homemoved.append(round(d,5))
                # move from the old patch and get to the new one
                patch = self.grid.givePatchEnPos(self.x , self.y)
                self.patch.removeGuest()
                self.homePatch.removeGuest()
                patch.putGuest()
                # Remove depleted patch from memory
                self.removeMemory(patch)
                self.patch = patch
                self.homePatch = patch
                homePatch = patch
                self.resmoves += 1 
            
        #----------------------------------------------------------------------------------------------------------
        
    def removeMemory(self, patch):
        # This method substracts the patch from memory 
        z = 0
        # search for the patch coordinates to remove it from memory 
        
        while z < len(self.memories):
            if (patch.x == self.memories[z][0])&(patch.y == self.memories[z][1]):
                del self.memories[z]
                break
            z = z + 1
        #----------------------------------------------------------------------------------------------------------
    
    def check_radius_forfood(self,r):
        
        cb_home = []
        dis=[]
#         num = []
        z= 0 
        while z < len(self.memories) :
            distanceHZ =  math.sqrt(pow(self.homePatch.x - self.memories[z][0],2) + pow(self.homePatch.y - self.memories[z][1],2))
            dis.append(distanceHZ)
            cb_home.append((distanceHZ )/self.memories[z][2] )

            z = z + 1 

        best_cb = np.argmin(cb_home)  ##get index of minimum cost/benefit
 ##use index to find the patch
        to_return = [self.memories[best_cb][0], self.memories[best_cb][1], dis[best_cb]]
        if dis[best_cb] <= r : 
            return to_return
        else :
            return None 
        
        
    
       
    def check_forhome_avoidoverlap(self,r) :
        cb = []
        num=[]
        diss=[]
#         ks=[]
#         z= 0 
        for z in range(len(self.memories)) :
            dis = math.sqrt(pow(self.homePatch.x - self.memories[z][0],2) + pow(self.homePatch.y - self.memories[z][1],2))
           ##get all cost/benefit which fall out of the leapfrog radius 
            if dis > (er * r) :
                diss.append(dis)
                cb.append(dis / self.memories[z][2] )
                num.append(z)
#                 ks.append()
                
        
        best_cb = np.argmin(cb)
        best_home = num[best_cb]
        
        return self.memories[best_home][0], self.memories[best_home][1], diss[best_cb]
        
    
    
   ############################### End of class forager ###############################
                                    
class Patch: # This class represents a patch with all functions and attributes
    # create a patch with a size k, a position, and a name
    def __init__(self, x, y, k, name):
        # This method is called for creating a new patch

        # initialize the variables
        self.x = x
        self.y = y
        self.k = k
        self.name = name
        self.ocupantes = 0
        #----------------------------------------------------------------------------------------------------------
        
    def nextState(self):
        # This function calculates the next state of the patch based on how its k has changed
        if self.ocupantes > 0:
            self.removeFood()
        #----------------------------------------------------------------------------------------------------------
        
    def putGuest(self):
        # This method is called when a guest jumps on the patch
        self.ocupantes = self.ocupantes + 1
        #----------------------------------------------------------------------------------------------------------
        
    def removeGuest(self):
        # This method is called when a guest leaves
        self.ocupantes = self.ocupantes - 1
        #----------------------------------------------------------------------------------------------------------
        
    def removeFood(self):
        """This method is used to substract the foods eaten from a patch"""
        if self.k > 0:
            self.k = self.k - self.ocupantes
        ##print("decrease food", self.name, self.k, self.ocupantes, self.x, self.y)
        #----------------------------------------------------------------------------------------------------------
        

In [ ]:
def main():
    """esta es la funcion principal donde comienza la ejecucion del
    programa
    """
    # This is the main function that executes the program
    

    # create directory for saving all the files
    os.mkdir(mainFolder)
    

    # set initial beta and iterate until the max value of beta
    for beta in betas :
        

        # create a subdirectory for each value of beta       
        os.mkdir(mainFolder+"/beta"+str(beta))

        #por cada beta provado obten un kmax
        # for every beta that has been tested get a kmax
        kmax = giveKmax(patches, beta)
        

        # for each beta try a range of number of foragers (from a starting value to a ceiling)
        for forager in foragers:
            # create a subdirectory for each one of the different number of foragers
            os.mkdir(mainFolder+"/beta"+str(beta)+"/foragers"+str(forager))
            
            for rad in fRadius :
                os.mkdir(mainFolder+"/beta"+str(beta)+"/foragers"+str(forager)+"/radius"+str(rad))

            # for each one of the runs pick a specific beta and number of foragers
            # run the program for a certain amount of iterations with the same initial conditions
                z = 0
                while z < runs:
                    print ("beta = "+str(beta)+" foragers = "+str(forager)+" radius=" +str(rad)+"corrida = "+str(z))
                    f=open(mainFolder+"/beta"+str(beta)+"/foragers"+str(forager)+"/radius"+str(rad)+"/"+str(z)+".txt", 'w')
                    fdis = open(mainFolder+"/beta"+str(beta)+"/foragers"+str(forager)+"/radius"+str(rad)+"/"+str(z)+"_dis.txt", 'w')
                
                    ################### here's where the whole program is executed #######
                    print ("---- grid" )
                    grid= Grid(forager, memory, patches, beta, kmax)
                    print ("----------------------------------------")
                    step = 0
                    # Main cycle of each iteration
                    while step < steps:
                        print("timestep", step)
                        # call the function for the next state of the grid, the foragers, and patches
                        result = grid.nextState(rad)
                        f.write( result )

                        step = step + 1
                    dis_output=grid.output()
                    fdis.write(dis_output)
                    f.close()
                    fdis.close()
                    ###########################################################################
                    z = z+1

            
#         fig = plt.figure()

#         ax1 = fig.add_subplot(2,2,1)
# #             fig1, ax1 = plt.subplots()

#         ax1.hist(all_movdis)
#         ax1.xscale('log')
#         all_movdis=[]
    #----------------------------------------------------------------------------------------------------------

def giveKmax(numero_de_patches, beta):
    ## kmax represents the patch with the highest amount of foods possible 
    ## this function generations an appropriate distribution around a good kmax
    # initialize the variables 
    z = 1
    norm = 0
    pkmax = 0
    # search for Kmax until you find it 
    while True:      
        norm = norm + pow(z , -beta)
        pkmax = numero_de_patches * pow(z , -beta)/norm        
        if pkmax  <= 1.0 :
            print ("kmax = "+str(z))
            return z 
        z =  z + 1
        #----------------------------------------------------------------------------------------------------------      





           
# if this module has a main, call it    
if __name__ == '__main__': main()
    

